In [1]:
import load_env
from elasticsearch import AsyncElasticsearch
from src.models.annotation_model import Annotation, AnnotationStats, Frequency, Bucket
from src.config.settings import settings
from src.config.es import  es


In [11]:

async def get_annotations_stats():
    
    search_query = {
            "match_all":{},
        }
    aggs = {
        "annotations_frequency": {
          "terms": {
            "field": "term.keyword",
             "order":{"_count":"desc"},
          }
        }
    }
    resp = await es.search(
          index=settings.PANTHER_ANNOTATIONS_INDEX,
          filter_path ='took,hits.total.value,aggregations',
          query=search_query,
          aggs=aggs,
          size=0,
    )

    buckets = [Bucket(**bucket) for bucket in resp['aggregations']['annotations_frequency']['buckets']]
    freq = Frequency(buckets=buckets)
    results = AnnotationStats(annotations_frequency=freq)
        
    return results

res = await get_annotations_stats()


res

AnnotationStats(annotations_frequency=Frequency(buckets=[Bucket(key='GO:0000981', doc_count=2), Bucket(key='GO:0000978', doc_count=1), Bucket(key='GO:0003677', doc_count=1), Bucket(key='GO:0003723', doc_count=1), Bucket(key='GO:0003823', doc_count=1), Bucket(key='GO:0003924', doc_count=1), Bucket(key='GO:0004527', doc_count=1), Bucket(key='GO:0004672', doc_count=1), Bucket(key='GO:0004984', doc_count=1), Bucket(key='GO:0005125', doc_count=1)]))